##**Atividade Avaliativa - Processamento de Linguagem Natural**
### **Equipe:** Ana Clara, Bruno, Igor e Pedro Cruz


# **Descrição e motivação do problema:**

A disseminação de **fake news** tem acontecido com cada vez mais frequência e possui impacto significativo na sociedade, contribuindo para a desinformação, polarização social e decisões equivocadas. A motivação para desenvolver um algoritmo de identificação de notícias falsas está em promover uma sociedade mais informada, fornecendo as pessoas uma ferramenta automatizada para avaliar a veracidade de informações rapidamente. Isso pode ajudar a mitigar os efeitos negativos da desinformação, capacitando as pessoas a tomarem decisões mais conscientes e baseadas em fatos confiáveis.

# **Descrição da base de dados (e do processo de obtenção dos dados, se for o caso):**

A base de dados que decidimos utilizar é a [Fake.Br Corpus](https://github.com/roneysco/Fake.br-Corpus). Ela fornece pares de notícias verdadeiras e falsas em português.

Mais especificamente, iremos utilizar os dados contidos na pasta "size_normalized_texts", que contém versões truncadas dos textos, onde em cada par verdadeiro-falso o texto mais longo é truncado (em número de palavras) para o tamanho do texto mais curto. Esta versão do corpus pode ser útil para evitar inclinações incorretas em experimentos de aprendizado de máquina.

# **Objetivo de negócio ou científico associado ao problema:**


O objetivo científico deste projeto é desenvolver e validar um algoritmo capaz de identificar fake news de algum assunto específico com precisão. Já o objetivo de negócio é fornecer uma solução automatizada, como um chatbot, que permita organizações, jornalistas e usuários individuais verificarem rapidamente a veracidade de informações, reduzindo os impactos da desinformação em escala e promovendo a confiança em fontes confiáveis.

# **Pré-processamento & Extração de Características:**

Unzipando os arquivos baixados

In [1]:
import zipfile

In [2]:
caminho_zip = 'Fake.br-Corpus-master.zip'

with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
  zip_ref.extractall('arquivos')

Obtendos os dados da pasta size_normalized_texts, como comentado anteriormente, e colocando tudo em um dataframe, inserindo o label 0 ou 1 para distinguirmos entre verdadeiro ou falso.

In [3]:
import os
import pandas as pd

In [4]:
fake_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/fake'
true_dir = 'arquivos/Fake.br-Corpus-master/size_normalized_texts/true'

def load_texts(directory, label):
    data = []
    for file in os.listdir(directory):
        with open(os.path.join(directory, file), 'r', encoding='utf-8') as f:
            data.append({'text': f.read(), 'label': label})
    return data

fake_data = load_texts(fake_dir, 0)  # 0 para "falso"
true_data = load_texts(true_dir, 1)  # 1 para "verdadeiro"

df = pd.DataFrame(fake_data + true_data)

print(df.head())

                                                text  label
0  A cor do luto será mudada pra cinza pra evitar...      0
1  Joaquim Barbosa disse pretende disputar a\nPre...      0
2  Você vai se arrepiar! Durante show de rock,  p...      0
3  URGENTE: Homem ameaça explodir prédio durante ...      0
4  EUA lançam míssil nuclear da base de North Van...      0


Exploração

In [5]:
df.shape

(7200, 2)

Verificando se temos algum nulo

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7200 non-null   object
 1   label   7200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.6+ KB


Pré-processamento

In [7]:
pip install transformers

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
import pickle

In [9]:
PRE_TRAINED_MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
MAX_LEN = 128

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
df.head()
def generate_embeddings_batchwise(texts, tokenizer, model, max_len, batch_size, device):
    encoded = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    dataset = TensorDataset(encoded['input_ids'], encoded['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=batch_size)

    embeddings = []
    with torch.no_grad():
        for input_ids, attention_mask in dataloader:
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_embedding.cpu().numpy())
    return np.vstack(embeddings)

In [ ]:
texts = df['text'].tolist()
labels = df['label'].tolist()

print("Gerando embeddings...")
batch_size = 32
embeddings = generate_embeddings_batchwise(texts, tokenizer, bert_model, MAX_LEN, batch_size, device)


Gerando embeddings...


# **Modelos de Machine Learning:**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

print("Treinando o classificador...")
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import classification_report

#vectorizer = TfidfVectorizer(max_features=5000)
#X = vectorizer.fit_transform(df['processed_text']).toarray()
#y = df['label']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#model = MultinomialNB()
#model.fit(X_train, y_train)

#y_pred = model.predict(X_test)
#print(classification_report(y_test, y_pred))

# **Protocolo de experimentos e validação**

In [ ]:
print("Avaliando o modelo...")
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['falso', 'verdadeiro']))

In [ ]:
CONFIDENCE_THRESHOLD = 0.7

def predict_with_threshold(text, tokenizer, model, classifier, max_len, device, threshold):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    probas = classifier.predict_proba(cls_embedding)[0]
    max_proba = np.max(probas)
    if max_proba < threshold:
        return "Não sei", max_proba
    return ("Verdadeiro" if np.argmax(probas) == 1 else "Falso"), max_proba

In [ ]:
test_sentences = [
    "EUA lançam míssil nuclear.",
    "Brasil declara guerra.",
    "Guitarrista atira no público durante show de rock."
]

for sentence in test_sentences:
    result, confidence = predict_with_threshold(
        sentence, tokenizer, bert_model, classifier, MAX_LEN, device, CONFIDENCE_THRESHOLD
    )
    print(f"Frase: {sentence}")
    print(f"Resposta: {result}, Confiança: {confidence:.2f}\n")

# **Discussão dos resultados e trabalhos futuros**